In [2]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-04 08:37:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-03-04 08:37:36 (11.6 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...| 2015-08-31|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend|           Books|          5|    

### Create DataFrames to match tables

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [7]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine','verified_purchase'])
vine_df.show(20)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RJOVP071AVAJO|          5|            0|          0|   N|                Y|
|R1ORGBETCDW3AI|          5|            0|          0|   N|                Y|
| R7TNRFQAOUTX5|          5|            0|          0|   N|                Y|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|                N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|                Y|
|R13U5PBJI1H94K|          2|            1|          1|   N|                N|
|R1H8UVH990F8VE|          5|            2|          2|   N|                N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|                Y|
| R2NO2HXK16Y4J|          5|            0|          0|   N|                Y|
|R245YIAVJK82ZL|          5|            0|          0|   N|     

### Analysis Portion

In [12]:
# 1 - total_votes > 20
# Filter the data and create a new DataFrame to retrieve all the rows where the total_votes count is equal 
# to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

# Model
# df1 = main_df.filter(main_df.total_votes > 20)
vine_over20_df = vine_df.filter(vine_df.total_votes >= 20)
vine_over20_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R2QAPWF1M5863L|          1|            5|         23|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [13]:
# 2 - 
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all
# the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.

# Model
# df1 = df2.filter((d1.columnA / d1.columnB) >= #)
vine_over50_df = vine_over20_df.filter((vine_over20_df.helpful_votes / vine_over20_df.total_votes) >= 0.50 )
vine_over50_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [ ]:
# 3 -
# Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all
# the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

# Model
# 


In [ ]:
# 4 -
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'.

# Model
# 


In [ ]:
# 5 -
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews
# for the two types of review (paid vs unpaid).

# Model
# 
